This is a test file for the FieldData class
It requires input parameters from the command line as follow


In [1]:
import sys

# check if GEE is already imported to avoid requesting authenticatiation multiple times
modulename = 'ee'
if modulename not in sys.modules: 
   # import GEE and Authenticate, token or log in will be asked from web browser
   import ee
   ee.Authenticate()
   ee.Initialize()
else:
   print('GEE already imported')
   # google earth engine already imported and authenticated


import sys
modulename = 'ipynb_FieldData'
if modulename not in sys.modules:
   %run FieldData.ipynb
   # adding an identifier to sys.modules to avoiding loading the same file multiple times
   sys.modules['FieldData'] = None 
#else
   # Utils modules has already been loaded somewhere else

modulename = 'ipynb_Sentinel2b'
if modulename not in sys.modules:
   %run Sentinel2b.ipynb
   # adding an identifier to sys.modules to avoiding loading the same file multiple times
   sys.modules['ipynb_Sentinel2b'] = None 
#else
   # Utils modules has already been loaded somewhere else

modulename = 'ipynb_MapVisualisation'
if modulename not in sys.modules:
   %run MapVisualisation.ipynb
   # adding an identifier to sys.modules to avoiding loading the same file multiple times
   sys.modules['ipynb_MapVisualisation'] = None 
#else
   # Utils modules has already been loaded somewhere else


GEE already imported


<>:27: SyntaxWarning: invalid escape sequence '\c'
C:\Users\mm1397\AppData\Local\Temp\ipykernel_37160\2623374617.py:27: SyntaxWarning: invalid escape sequence '\c'
  raise Exception("CSV input file should be defined as follow  '\csvfilename'\:<csvfilename>")


Class fieldData imported
Utils imported
Masks class imported
Sentinel2b class imported
MapVisualisation class imported - uses folium library


In [3]:
# parsing command line inputs - Could not find a way to add the arguments in ipython
""" parser = argparse.ArgumentParser()
parser.add_argument("-in",
     required=True,
     help="The input csv file with its directory containing field plot data",
     metavar='<string>')



params       = vars(parser.parse_args())
csvFile      = params["in"   ]


print ("imported csv file     = ", csvFile ) """

# import a Sentinel 2 collection and crop Spain
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
AOI = countries.filter(ee.Filter.eq('country_na', 'Spain'))

geometry = ee.Geometry.Polygon(
    [[[-5.586640856054208, 40.07176423657564],
      [-5.391633531835458, 40.07176423657564],
      [-5.391633531835458, 40.17257546939766],
      [-5.6106734488276455, 40.170476760191924]]])


START_DATE = '2021-01-01'
END_DATE = '2021-01-28'
masks = {}#'gsw': 30, 'lmask': 30, 'forestMask': {30,2021}}


s2_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(geometry) \
            .filterDate('2021-01-01', '2021-12-31') \
            .median()

bandNames = s2_col.bandNames()

bandnames = s2_col.bandNames()
bandnames = bandnames.getInfo()
        
print(type(bandnames))
print('Band names s2_col: ', bandnames)



sentinel2Info = {
    "identifier": 'sentinel-2',                                              
    "collection": "COPERNICUS/S2_SR_HARMONIZED",
    #"launchDate" : "2015-05-23",
    #"retirmentDate":todayDate, #not retired yet
    "startDate"   : START_DATE,
    "endDate"     : END_DATE,
    #"added"     : False,
    "clouds"    : 50,
    "selectedBand": ['B8', 'B4'],
    "selectedIndices": ['ndvi']
} 


s2 = Sentinel2(AOI,sentinel2Info, masks)
s2.byMonth(2021)



# Create ee.Geometry.Point objects for the corners
lower_left_point = ee.Geometry.Point(
                        [292002.67, 4448723.55],"EPSG:3042"
                        )
upper_right_point =ee.Geometry.Point(
                        [292402.67, 4449073.55],"EPSG:3042"
                        )

# Construct a rectangle from the list of ee.Geometry.Point objects
polygon =  ee.Geometry.Rectangle([lower_left_point, upper_right_point])


polygon = ee.FeatureCollection([ee.Feature(ee.Geometry.Polygon(polygon.coordinates()))])


# Coordinates of the rectangle's lower-left and upper-right corners
x_min, y_min = 292002.67, 4448723.55
x_max, y_max = 292402.67, 4449073.55

MinPoint = ee.Geometry.Point([292002.67, 4448723.55], "EPSG:3042")
MaxPoint = ee.Geometry.Point([292402.67, 4449073.55], "EPSG:3042")

# Transform the point to latitudes and longitudes (WGS84)
MinPoint_latlon = MinPoint.transform("EPSG:3857")
MaxPoint_latlon = MaxPoint.transform("EPSG:3857")

# Get the coordinates as a list
coordinatesMinPoint = MinPoint_latlon.getInfo()["coordinates"]
coordinatesMaxPoint = MaxPoint_latlon.getInfo()["coordinates"]

# Extract latitude and longitude
latitudeMinPoint, longitudeMinPoint = coordinatesMinPoint[1], coordinatesMinPoint[0]
latitudeMaxPoint, longitudeMaxPoint = coordinatesMaxPoint[1], coordinatesMaxPoint[0]

csvFile = "./samplePlots.csv"


# Create a dictionary that holds the relevant plot information
fieldData = {
              "csvfilename"   : csvFile,
              "proj"          : "EPSG:3042",
              "radius"        : 200,
              "xcol"          : "CX",    
              "ycol"          : "CY",
              "outPlotFileWithIDs"  :"fieldDataWithIdentifiers/FieldDataTest.csv"
       }

csvDF = FieldData(fieldData)
# csvDF.filterYearsOfInterest(2017,2018,"year")

# create a list of polygons for the field data 1-100
bufferredPoints = csvDF.createBufferedPoints(1, 100)
#print(bufferredPoints)

s2bands = s2.getCollectionToBands()

csvDF.exportFeaturesMeanStdCSV(s2bands,"ouutCsvFeatureVectors2021_200_B4","GEE")



# THIS IS THE OTHER METHOD THAT EXPORTS FOR EACH PIXEL - DOES NOT HANDLES GAPS SO NEEDS EXTRA POST-PROCESSSING
# csvDF.exportfeatureVectorsToDrive(col,"TestIndClassSentinel2c.csv","earth_engine_demos",10)
# csvDF.mergeFeatureVectorsToPlotData("/home/milto/Documents/FLFScripts/sampleData/mergedcsv.csv")
# csvDF.exportPlotDataWithAddedIdentifiers("/home/milto/Documents/FLFScripts/sampleData/fieldDatawithIds.csv")

print("NOTE: in the example, there are no plots within the study area!")
print("   ***   EXIT   ***")


<class 'list'>
Band names s2_col:  ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE']


KeyError: 'allBands'

In [ ]:
#%run MapVisualisation.ipynb
my_map = folium.Map(location=[40,-3], zoom_start=5, height=400)
# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)
# Add Land Mask to the map
my_map.add_ee_layer(s2_col, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1}, 'median'      )
my_map.add_ee_layer(bufferredPoints, {},'bufferredPoints')

my_map.add_ee_layer(geometry,{},"geometry")

my_map.add_ee_layer(lower_left_point,{},"lower_left_point")

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())
plugins.Fullscreen().add_to(my_map)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)